In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [57]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential

In [58]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/prabuddhachowdhury/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [59]:
data = pd.read_csv('train.csv')

In [60]:
data

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [61]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [62]:
data.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [63]:
data.shape

(20800, 5)

In [64]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [65]:
data.dropna()
data.reset_index(inplace=True)

In [66]:
X = data.drop("label",axis=1)
y = data["label"]

In [67]:
print(X.shape)
print(y.shape)

(20800, 5)
(20800,)


In [68]:
messages=X.copy()
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
    review = str(messages['title'][i]) # convert to string
    review = re.sub("[^a-zA-Z]"," ",review)
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]  # (word not in stopwords) and stem(word)
    review = ' '.join(review)    
    corpus.append(review)


### One Hot Encoding

In [69]:
vocab_size=5000
onehot_repr=[one_hot(words,vocab_size)for words in corpus] 

In [70]:
onehot_repr

[[2600, 3896, 4587, 158, 4649, 2980, 4072, 118, 2277, 570],
 [1762, 1518, 3318, 3585, 4817, 908, 2551],
 [27, 4304, 4965, 838],
 [1394, 4904, 2613, 2081, 3965, 4045],
 [782, 4817, 4590, 296, 802, 44, 4817, 1472, 855, 4990],
 [4965,
  1646,
  755,
  4684,
  1120,
  1403,
  603,
  3326,
  4425,
  3531,
  2842,
  3595,
  3324,
  470,
  2551],
 [4343, 4343, 2189, 1540, 2761, 721, 491, 2130, 3105, 3115, 1244, 2288],
 [4704, 757, 2725, 2975, 1448, 2741, 86, 1176, 2885, 2858, 2331],
 [520, 223, 2592, 821, 1403, 2650, 2417, 2368, 771, 929, 2885, 2858, 2331],
 [2082, 2879, 4818, 4086, 4906, 3380, 1403, 3511, 2885, 2858, 2331],
 [2482, 3917, 3945, 3743, 1594, 1927, 4658, 1855, 1403, 3491],
 [1175, 1571, 199, 1118, 646, 2371, 1293, 3163],
 [2759, 2344, 4885, 3724, 1675, 4531, 150, 2646, 4209, 1760, 1237],
 [2081, 1949, 4649, 1927, 1403, 4906],
 [708, 890, 3480, 2154, 2351, 4356, 1424, 3660, 2895],
 [532, 128, 4711, 1604, 688, 2169, 1146, 2885, 2858, 2331],
 [4200, 3886, 2729, 4109, 4025, 2885, 28

In [71]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  118 2277  570]
 [   0    0    0 ... 4817  908 2551]
 [   0    0    0 ... 4304 4965  838]
 ...
 [   0    0    0 ... 2885 2858 2331]
 [   0    0    0 ...  268 4052 2554]
 [   0    0    0 ...  751 1536  961]]


In [72]:
# Creating Model for training set
embedding_vector_features=40
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [73]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [74]:
X_final

array([[   0,    0,    0, ...,  118, 2277,  570],
       [   0,    0,    0, ..., 4817,  908, 2551],
       [   0,    0,    0, ..., 4304, 4965,  838],
       ...,
       [   0,    0,    0, ..., 2885, 2858, 2331],
       [   0,    0,    0, ...,  268, 4052, 2554],
       [   0,    0,    0, ...,  751, 1536,  961]], dtype=int32)

In [75]:
y_final

array([1, 0, 1, ..., 0, 1, 1])

In [76]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [77]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=64)

Epoch 1/100
218/218 [==============================] - 3s 11ms/step - loss: 0.3116 - accuracy: 0.8532 - val_loss: 0.1875 - val_accuracy: 0.9247
Epoch 2/100
218/218 [==============================] - 2s 10ms/step - loss: 0.1288 - accuracy: 0.9496 - val_loss: 0.1913 - val_accuracy: 0.9216
Epoch 3/100
218/218 [==============================] - 2s 10ms/step - loss: 0.0905 - accuracy: 0.9670 - val_loss: 0.2300 - val_accuracy: 0.9260
Epoch 4/100
218/218 [==============================] - 2s 10ms/step - loss: 0.0630 - accuracy: 0.9791 - val_loss: 0.2504 - val_accuracy: 0.9222
Epoch 5/100
218/218 [==============================] - 2s 10ms/step - loss: 0.0390 - accuracy: 0.9881 - val_loss: 0.2633 - val_accuracy: 0.9164
Epoch 6/100
218/218 [==============================] - 2s 10ms/step - loss: 0.0232 - accuracy: 0.9929 - val_loss: 0.3760 - val_accuracy: 0.9119
Epoch 7/100
218/218 [==============================] - 2s 10ms/step - loss: 0.0174 - accuracy: 0.9942 - val_loss: 0.3481 - val_accuracy:

In [78]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

215/215 [==============================] - 1s 2ms/step


array([[3155,  294],
       [ 300, 3115]])

In [79]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9134615384615384